## Compton calculations
Notebook to calculate Compton rates, asymmetries, etc.
Default units are cm and MeV

In [ ]:
import numpy as np
import scipy.integrate as integrate
import matplotlib.pyplot as plt

In [ ]:
#Define some constants/parameters
Me=0.51099006
re=2.818E-13
hbarc=1.9732858E-11
e=1.6022E-19
c=2.9979E10
pi=3.141592653589793
# Laser parameters
laser_lambda=532E-7
sigmalaser=100E-4
siglaserz=23E-12*c
alphac=10E-3
alpha=alphac/2.0
plaser = 5
flas=100E6
# Beam parameters - nominal EIC
Ebeam=5000
#Ebeam=10000
#Ebeam=18000
Ibeam=1
sigmabeam=300E-4
sigbeamz = 23E-12*c
fbeam=100E6
fcoll=78000
#fcoll=2

In [ ]:
# Determine required size of entrance window for laser
# IP6 Compton: laser entrance window ~ 6 meters from interaction point
# window radius should be about 3.5 times larger than sigma_laser at window to ensure less than 0.1% power loss
# and allow a little room for misalignment
z_window = 600.0
Z_R = pi*(2*sigmalaser)**2/laser_lambda
sigma_window = sigmalaser*np.sqrt(1+(z_window/Z_R)**2)
print('Laser size at entrance window (cm) ', sigma_window)
print('Required window size (radius, cm) ', 3.5*sigma_window)

In [ ]:
Elaser = hbarc*2*pi/laser_lambda
gam = Ebeam/Me
a = 1.0/(1 + 4*gam*Elaser/Me)
Egmax = 4*gam**2*a*Elaser
Thetahalf = np.sqrt(1/(a*gam**2))
#thetagamma = np.sqrt((Elaser/(rho*Egmax)*4.0*a*gam**2 - 1)/(a*gam**2))
#rho = Eg/Egmax 
rho0 = 1/(1 + a)
Pbeam = np.sqrt(Ebeam**2 - Me**2) 
rhomin=Elaser/Egmax
print('Laser energy (MeV): ',Elaser)
print('Maximum backscattered photon energy (MeV): ',Egmax)
print('Photon cone half-angle (deg.): ',Thetahalf*180/pi)
print('Rho at zero crossing: ',rho0)

In [ ]:
#Define some functions
def compton_xsec(rho):
 first = rho**2*(1.0-a)**2/(1.0-rho*(1.-a))
 second = (1-rho*(1+a))/(1-rho*(1-a))
 dsig = 2.*pi*re**2*a*(first+1.0+second**2)
 return dsig

def compton_asy_par(rho):
 first = rho**2*(1.0-a)**2/(1.0-rho*(1.-a))
 second = (1-rho*(1+a))/(1-rho*(1-a))

 term1 = 1.0/(first+1.0+second**2)
 term2 = 1.0-rho*(1+a)
 term3 = 1.0-1.0/(1.0-rho*(1.-a))**2

 asy=term1*term2*term3

 return asy

def compton_asy_perp(rho):
    # this is the phi-independent term - away from electron polarization direction, need *cos(phi)
 first = rho**2*(1.0-a)**2/(1.0-rho*(1.-a))
 second = (1-rho*(1+a))/(1-rho*(1-a))

 term1 = 1.0/(first+1.0+second**2)
 term2 = rho*(1-a)
 term3 = np.sqrt(4*a*rho*(1-rho))/(1-rho*(1-a))

 asy = term1*term2*term3
 return asy
 

In [ ]:
#Calculate luminosity, cross section
Ne = Ibeam/e/c
Nphot = plaser/(e*1E6*Elaser*c)
LumiCW = c*(1.0 + np.cos(alphac))*Ne*Nphot/(np.sin(alphac)*np.sqrt(2*pi)*np.sqrt(sigmabeam**2 + sigmalaser**2))
print('Luminosity for CW laser/beam (small crossing angle): ', LumiCW)
Nep = Ibeam/e/fbeam
Nphotp = plaser/(e*1E6*Elaser*flas)
LumiP = flas*Nep*Nphotp*np.cos(alpha)/2/pi*1/np.sqrt(sigmabeam**2+sigmalaser**2)*1/np.sqrt((sigmabeam**2+sigmalaser**2)*np.cos(alpha)**2+(siglaserz**2+sigbeamz**2)*np.sin(alpha)**2)
print('Luminosity for Pulsed laser/beam (small crossing angle): ', LumiP)
LumiSingleBunch = fcoll*Nep*Nphotp*np.cos(alpha)/2/pi*1/np.sqrt(sigmabeam**2+sigmalaser**2)*1/np.sqrt((sigmabeam**2+sigmalaser**2)*np.cos(alpha)**2+(siglaserz**2+sigbeamz**2)*np.sin(alpha)**2)
print('Luminosity for Pulsed laser colliding with one beam bunch (small crossing angle): ', LumiSingleBunch)
xsect = integrate.quad(lambda rho: compton_xsec(rho),rhomin,1.0)
rate = xsect[0]*LumiSingleBunch
print('Backscattered photon rate (Hz)', rate)

In [ ]:
rr = np.arange(0,1.0,0.01)
plt.xlabel(r'$\rho=E_{\gamma}/E^\mathrm{max}_{\gamma}$',fontsize=18)
plt.ylabel(r'$\frac{d\sigma}{d\rho}$',fontsize=18)
plt.plot(rr, compton_xsec(rr))

In [ ]:
rr = np.arange(0,1.0,0.01)
plt.xlabel(r'$\rho=E_{\gamma}/E^\mathrm{max}_{\gamma}$',fontsize=18)
plt.ylabel(r'$A_\mathrm{par}$',fontsize=18)
plt.plot(rr, compton_asy_par(rr))

In [ ]:
plt.xlabel(r'$\rho=E_{\gamma}/E^\mathrm{max}_{\gamma}$',fontsize=18)
plt.ylabel(r'$A_\mathrm{perp}$',fontsize=18)
plt.plot(rr, compton_asy_perp(rr))

In [ ]:
# Calculate measurement times
# need to define desired statistical precision (dP/P) and polarization
dP=0.01
Pe=0.8

In [ ]:
# Average asymmetry (longitudinal)
num = integrate.quad(lambda rho: compton_asy_par(rho)*compton_xsec(rho),rhomin,1.0)
A_avg = num[0]/xsect[0]
t_avg = 1.0/(rate*dP**2*Pe**2*A_avg**2)
print('Average longitudinal asymmetry: ', A_avg)
print('Time for 1% measurement (s): ', t_avg)

In [ ]:
# Differential measurement (longitudinal), avg. of A**2
num = integrate.quad(lambda rho: (compton_asy_par(rho))**2*compton_xsec(rho),rhomin,1.0)
A2_avg = num[0]/xsect[0]
t_avg = 1.0/(rate*dP**2*Pe**2*A2_avg)
print('sqrt(Average asymmetry**2): ', np.sqrt(A2_avg))
print('Time for 1% measurement (s): ', t_avg)

In [ ]:
# Energy-weighted asymmetry (long)
num = integrate.quad(lambda rho: rho*(compton_asy_par(rho))*compton_xsec(rho),rhomin,1.0)
den= integrate.quad(lambda rho: rho*compton_xsec(rho),rhomin,1.0)
EgammaA_avg = num[0]/den[0]
t_avg = 1.0/(rate*dP**2*Pe**2*EgammaA_avg**2)
print('sqrt(Average asymmetry**2): ', np.sqrt(A2_avg))
print('Time for 1% measurement (s): ', t_avg)

In [ ]:
# Plot asymmetry vs. y position at detector (phi=0 only)
# theta = sqrt[(1-rho)/(a*rho)]/gamma
def compton_ydet(rho): 
# detector about 25 m away    
 zdet = 25.0*100
 theta_gamma = np.sqrt((1-rho)/(a*rho))/gam
 ydet = zdet*np.tan(theta_gamma)
 return ydet

rr = np.arange(0.005,1,0.005)
plt.ylabel(r'$y_\mathrm{det} (\mathrm{cm})$',fontsize=18)
plt.xlabel(r'$\rho$',fontsize=18)

plt.plot(rr, compton_ydet(rr))


In [ ]:
# Plot asymmetry vs. y position at detector (phi=0 only) to estimate needed photon detector size
# use simple, approximate formula for lab scattering angle
# theta = sqrt[(1-rho)/(a*rho)]/gamma
def compton_asy_ydet(ydet): 
# detector about 25 m away    
 zdet = 25.0*100
 theta_gamma = np.arctan(abs(ydet)/zdet)
 rho = 1/(a*gam**2*theta_gamma**2 + 1)

 first = rho**2*(1.0-a)**2/(1.0-rho*(1.-a))
 second = (1-rho*(1+a))/(1-rho*(1-a))
 dsig = 2.*pi*re**2*a*(first+1.0+second**2)
    
 term1 = 1.0/(first+1.0+second**2)
 term2 = rho*(1-a)
 term3 = np.sqrt(4*a*rho*(1-rho))/(1-rho*(1-a))

 asy = ydet/abs(ydet)*term1*term2*term3
 return asy

rr = np.arange(-2,2,0.005)
plt.xlabel(r'$y_\mathrm{det} (\mathrm{cm})$',fontsize=18)
plt.ylabel(r'$A_\mathrm{perp}$',fontsize=18)

plt.plot(rr, compton_asy_ydet(rr))


In [ ]:
# Determine detector size for electron detector
# assume ~3 m dipole with 5 m drift to an electron detector, 0.8 degree bend at nominal energy
# ignore scattering angle of electrons, just treat the dispersion induced by the dipole 
def edet_pos(rho):
    zdrift=500.0
    Lmag=300.0
    thnom = 0.8*pi/180
    
    Rnom=Lmag/thnom
    Bnom=(10/3)*Ebeam/Rnom
    
    E_elec = Ebeam-rho*Egmax
    theta_e = np.arcsin(0.3*Bnom*Lmag/E_elec)
    
    xdip = 10/3*(Ebeam/Bnom)*(1-np.cos(theta_e))
    xdrift=zdrift*np.tan(theta_e)
    xtot = xdip+xdrift
    
    return xtot

# displacement at beam energy (no scattering)
xnom = edet_pos(0.0)
# displacement at Compton endpoint (rho=1)
xmax = edet_pos(1.0)
# displacement at zero-crossing
xzero = edet_pos(rho0)
# relative displacements
print('Scattered electron displacement at endpoint (cm) ', xmax-xnom)
print('Scattered electron displacement at asymmetry zero-crossing (cm) ', xzero-xnom)